In [ ]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
from alpha_vantage.fundamentaldata import FundamentalData
import os

# Initialize Alpha Vantage API Key and FundamentalData
api_key = 'KUH3E7WHSUQL16RD'
fd = FundamentalData(key=api_key, output_format='pandas')

# Example list of tickers
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'ZM']

# Fetch data from Yahoo Finance
def fetch_data(ticker):
    try:
        return yf.download(ticker, start='2020-01-01', end='2024-01-01')
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None
        
# Technical Analysis
def perform_technical_analysis(data):
    close = data['Close']
    high = data['High']
    low = data['Low']
    volume = data['Volume']

    # Relative Strength Index (RSI)
    rsi = talib.RSI(close, timeperiod=14)

    # Moving Averages - Simple and Exponential
    sma50 = talib.SMA(close, timeperiod=50)
    ema50 = talib.EMA(close, timeperiod=50)

    # Moving Average Convergence Divergence (MACD)
    macd, macd_signal, _ = talib.MACD(close)

    # Bollinger Bands
    upper_band, middle_band, lower_band = talib.BBANDS(close)

    # Stochastic Oscillator
    stoch_k, stoch_d = talib.STOCH(high, low, close)

    # Average True Range (ATR) for Volatility
    atr = talib.ATR(high, low, close, timeperiod=14)

    # Commodity Channel Index (CCI)
    cci = talib.CCI(high, low, close, timeperiod=20)

    # On-Balance Volume (OBV)
    obv = talib.OBV(close, volume)

    # Fibonacci Retracement Levels (here we calculate for the last 100 data points)
    max_high = np.max(high[-100:])
    min_low = np.min(low[-100:])
    fibonacci_retracement_levels = {
        '23.6%': max_high - 0.236 * (max_high - min_low),
        '38.2%': max_high - 0.382 * (max_high - min_low),
        '50%': max_high - 0.5 * (max_high - min_low),
        '61.8%': max_high - 0.618 * (max_high - min_low),
        '78.6%': max_high - 0.786 * (max_high - min_low),
    }

    # Accumulation/Distribution Line (ADL)
    adl = talib.AD(close, high, low, volume)

    # Compile all indicators into a dictionary
    indicators = {
        'rsi': rsi.iloc[-1],
        'sma50': sma50.iloc[-1],
        'ema50': ema50.iloc[-1],
        'macd': macd.iloc[-1],
        'macd_signal': macd_signal.iloc[-1],
        'upper_band': upper_band.iloc[-1],
        'middle_band': middle_band.iloc[-1],
        'lower_band': lower_band.iloc[-1],
        'stoch_k': stoch_k.iloc[-1],
        'stoch_d': stoch_d.iloc[-1],
        'atr': atr.iloc[-1],
        'cci': cci.iloc[-1],
        'obv': obv.iloc[-1],
        'adl': adl.iloc[-1],
        # Exclude the fibonacci_retracement_levels from indicators
    }

    return indicators


# Fundamental Analysis using Alpha Vantage
def fetch_fundamental_data(ticker):
    try:
        data = {}
        
        # Fetching required reports
        income_statement, _ = fd.get_income_statement_annual(ticker)
        balance_sheet, _ = fd.get_balance_sheet_annual(ticker)
        cash_flow, _ = fd.get_cash_flow_annual(ticker)
        overview, _ = fd.get_company_overview(ticker)

        # Extracting data for the latest year
        latest_income_statement = income_statement.iloc[:, 0]
        latest_balance_sheet = balance_sheet.iloc[:, 0]
        latest_cash_flow = cash_flow.iloc[:, 0]
        latest_overview = overview.iloc[:, 0]

        # Create a dictionary to store fundamental items
        fundamental_data = {
            'EPS': latest_income_statement.get('reportedEPS', None),
            'P/E Ratio': latest_overview['PERatio'],
            'P/S Ratio': latest_overview['PriceToSalesRatioTTM'],
            'P/B Ratio': latest_overview['PriceToBookRatio'],
            'Debt/Equity Ratio': total_debt / total_equity if total_equity != 0 else None,
            'PEG Ratio': latest_overview['PEGRatio'],
            'Free Cash Flow': latest_cash_flow['operatingCashflow'] - latest_cash_flow['capitalExpenditures'],
            'Dividend Yield': latest_overview['DividendYield'],
            'Dividend Payout Ratio': latest_overview['PayoutRatio'],
            'Return on Equity': latest_income_statement['netIncome'] / total_equity if total_equity != 0 else None
        }

        return fundamental_data

    except Exception as e:
        print(f"Error fetching fundamental data for {ticker}: {e}")
        return {}


# Sentiment and Correlation Analysis (Placeholders)
def perform_sentiment_analysis(ticker):
    return np.random.rand()

def perform_correlation_analysis(data, market_data):
    return np.random.rand()

# Fetch market data for a list of tickers
def fetch_market_data(tickers):
    market_data = []
    for ticker in tickers:
        stock_data = fetch_data(ticker)
        if stock_data is not None:
            ta_indicators = perform_technical_analysis(stock_data)
            numeric_indicators = {k: v for k, v in ta_indicators.items() if isinstance(v, (int, float))}
            ta_score = np.mean(list(numeric_indicators.values())) if numeric_indicators else np.nan

            sentiment_score = perform_sentiment_analysis(ticker)
            correlation_score = perform_correlation_analysis(stock_data, market_data)

            if not np.isnan(ta_score) and not np.isnan(sentiment_score) and not np.isnan(correlation_score):
                total_score = np.mean([ta_score, sentiment_score, correlation_score])
                # Create a dictionary for the stock's results
                stock_results = {
                    'Ticker': ticker,
                    'TA Score': ta_score,
                    'Sentiment Score': sentiment_score,
                    'Correlation Score': correlation_score,
                    'Total Score': total_score
                }
                # Fetch fundamental data
                fundamental_data = fetch_fundamental_data(ticker)
                stock_results.update(fundamental_data)  # Add fundamental data to the dictionary
                market_data.append(stock_results)
            else:
                print(f"Insufficient data to calculate score for {ticker}")

    return pd.DataFrame(market_data)  # Convert the list of dictionaries to a DataFrame

# Fetching market data for the listed tickers
results_df = fetch_market_data(tickers)

# Sort the DataFrame by Total Score in descending order
results_df.sort_values(by='Total Score', ascending=False, inplace=True)

# Display the DataFrame
print("Stock Analysis:")
print(results_df)